##

## Confusion matrix

In [3]:
import pandas as pd
csv_data = pd.read_csv('outputs/run_1/cam1_detections.csv', skiprows=[0], header=None)
y_predictions = csv_data[1].to_numpy()

In [4]:
csv_data[1].shape

(166,)

In [5]:
import numpy as np
y_expected = np.full((166,), 'Luis')

In [6]:
y_predictions, y_expected

(array([' David', 'Luis', 'Luis', 'Luis', 'Luis', 'Luis', 'Luis', 'Luis',
        'Luis', 'Luis', 'Luis', 'Luis', 'Luis', 'Luis', ' Patricio',
        ' Juan', 'Luis', ' David', 'Luis', ' Roberto', ' Julio', ' David',
        ' David', ' Patricio', ' David', ' David', ' David', ' David',
        ' David', 'Luis', 'Luis', ' David', ' Julio', ' Patricio',
        ' David', 'Luis', ' David', ' Juan', 'Luis', ' Patricio', 'Luis',
        'Luis', 'Luis', 'Luis', 'Luis', ' Juan', ' Juan', ' Roberto',
        ' Roberto', ' Juan', ' Roberto', ' Julio', ' David', ' David',
        ' Julio', ' Julio', ' David', ' Julio', ' Julio', ' Roberto',
        ' Julio', ' Julio', ' Juan', ' Patricio', 'Luis', ' Patricio',
        'Luis', 'Luis', 'Luis', ' Patricio', ' Julio', 'Luis', 'Luis',
        ' Roberto', ' Patricio', 'Luis', 'Luis', 'Luis', 'Luis', ' Julio',
        ' Julio', ' Julio', ' Julio', ' Julio', ' Julio', ' Julio',
        ' Julio', ' Julio', ' Julio', ' Roberto', ' Roberto', ' Patricio',

## Doing an array with detection results

In [7]:
y_expected_binary = np.full((166,), True).astype(int)

In [8]:
y_predictions_binary = (y_expected == y_predictions).astype(int)

In [125]:
y_expected_binary[0] = 0

In [9]:
y_predictions_binary

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [10]:
y_expected_binary

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [15]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(y_expected_binary.astype(int), y_predictions_binary.astype(int)).ravel()

In [16]:
tn, fp, fn, tp

(0, 0, 113, 53)

# Implementing models with Keras

In [1]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

In [1]:
parent_root = 'Datasets/Propio/primer_grupo/test_data'
target_root = 'Datasets/espe/masked/test'
csv_path = 'Datasets/espe/masked/test/test.csv'

from processing import generate_masked_dataset

generate_masked_dataset(parent_root, target_root, csv_path)

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead
Working on:  Datasets/Propio/primer_grupo/test_data\0
Opening  Datasets/Propio/primer_grupo/test_data\0\person_0_241.jpg
Processing person_0_1.jpg ...
Saving file on Datasets/espe/masked/test\0\person_0_1.jpg
Opening  Datasets/Propio/primer_grupo/test_data\0\person_0_242.jpg
Processing person_0_2.jpg ...
Saving file on Datasets/espe/masked/test\0\person_0_2.jpg
Opening  Datasets/Propio/primer_grupo/test_data\0\person_0_243.jpg
Processing person_0_3.jpg ...
Saving file on Datasets/espe/masked/test\0\person_0_3.jpg
Opening  Datasets/Propio/primer_grupo/test_data\0\person_0_244.jpg
Processing person_0_4.jpg ...
Saving file on Datasets/espe/masked/test\0\person_0_4.jpg
Opening  Datasets/Propio/primer_grupo/test_data\0\person_0_245.jpg
Processing

In [3]:
parent_root = 'Datasets/espe/base/validation'
target_root = 'Datasets/espe/lbp_histograms - copia/validation'
csv_path = 'Datasets/espe/lbp_histograms - copia/validation/validation.csv' # areas and perimeters csv dataset path
from processing import generate_dataset_with_lbp
generate_dataset_with_lbp(parent_root, target_root, csv_path)

Working on:  Datasets/espe/base/validation\0
Processing person_0_1.jpg ...
Opening  Datasets/espe/base/validation\0\001_9 14.jpg
Saving file on Datasets/espe/lbp_histograms - copia/validation\0\person_0_1.jpg
Processing person_0_2.jpg ...
Opening  Datasets/espe/base/validation\0\001_9 15.jpg
Saving file on Datasets/espe/lbp_histograms - copia/validation\0\person_0_2.jpg
Processing person_0_3.jpg ...
Opening  Datasets/espe/base/validation\0\001_9 18.jpg
Saving file on Datasets/espe/lbp_histograms - copia/validation\0\person_0_3.jpg
Processing person_0_4.jpg ...
Opening  Datasets/espe/base/validation\0\001_9 20.jpg
Saving file on Datasets/espe/lbp_histograms - copia/validation\0\person_0_4.jpg
Processing person_0_5.jpg ...
Opening  Datasets/espe/base/validation\0\001_9 27.jpg
Saving file on Datasets/espe/lbp_histograms - copia/validation\0\person_0_5.jpg
Processing person_0_6.jpg ...
Opening  Datasets/espe/base/validation\0\001_9 28.jpg
Saving file on Datasets/espe/lbp_histograms - copia

In [17]:
from processing import generate_masks_datasets

parent_root = 'Datasets/Propio/primer_grupo/train_data'
target_root = 'Datasets/espe/masks/train'

generate_masks_datasets(parent_root, target_root)

Working on:  Datasets/Propio/primer_grupo/train_data\0
Processing person_0_1.jpg ...
Opening  Datasets/Propio/primer_grupo/train_data\0\person_0_1.jpg
Saving file on Datasets/espe/masks/train\0\person_0_1.jpg
Processing person_0_2.jpg ...
Opening  Datasets/Propio/primer_grupo/train_data\0\person_0_10.jpg
Saving file on Datasets/espe/masks/train\0\person_0_2.jpg
Processing person_0_3.jpg ...
Opening  Datasets/Propio/primer_grupo/train_data\0\person_0_100.jpg
Saving file on Datasets/espe/masks/train\0\person_0_3.jpg
Processing person_0_4.jpg ...
Opening  Datasets/Propio/primer_grupo/train_data\0\person_0_101.jpg
Saving file on Datasets/espe/masks/train\0\person_0_4.jpg
Processing person_0_5.jpg ...
Opening  Datasets/Propio/primer_grupo/train_data\0\person_0_102.jpg
Saving file on Datasets/espe/masks/train\0\person_0_5.jpg
Processing person_0_6.jpg ...
Opening  Datasets/Propio/primer_grupo/train_data\0\person_0_103.jpg
Saving file on Datasets/espe/masks/train\0\person_0_6.jpg
Processing p

In [1]:
# generating market dataset - LBP
from processing import generate_dataset_with_lbp

parent_root = 'Datasets/Market-1501/rest'
target_root = 'Datasets/Market-1501/rest-lbp'
csv_path = 'Datasets/Market-1501/rest-lbp/train.csv'

generate_dataset_with_lbp(parent_root, target_root, csv_path)

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead
Working on:  Datasets/Market-1501/rest\0895
Processing person_0895_1.jpg ...
Opening  Datasets/Market-1501/rest\0895\0895_c1s4_049681_04.jpg
Copying  image ...
Detecting people ...
Calculating mask area and perimeter ...
Saving file on Datasets/Market-1501/rest-lbp\0895\person_0895_1.jpg
Processing person_0895_2.jpg ...
Opening  Datasets/Market-1501/rest\0895\0895_c1s4_049706_03.jpg
Copying  image ...
Detecting people ...
Calculating mask area and perimeter ...
Saving file on Datasets/Market-1501/rest-lbp\0895\person_0895_2.jpg
Processing person_0895_3.jpg ...
Opening  Datasets/Market-1501/rest\0895\0895_c1s4_049756_03.jpg
Copying  image ...
Detecting people ...
Calculating mask area and perimeter ...
Saving file on Datasets/Market-1501/rest